In [1]:
import pandas as pd
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import os, glob
from PIL import Image
from skimage import io, color
import cv2 
%matplotlib inline

import tensorflow as tf
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout
from keras.layers import MaxPooling2D,Flatten
from keras.callbacks import ModelCheckpoint,TensorBoard

In [2]:
cd = os.getcwd()


In [3]:
Xtrain = pd.read_csv(r'C:\Users\User\Desktop\TrainTestData\Xtrain_data.csv')
ytrain = pd.read_csv(r'C:\Users\User\Desktop\TrainTestData\ytrain.csv')
Xtest = pd.read_csv(r'C:\Users\User\Desktop\TrainTestData\Xtest_data.csv')
ytest = pd.read_csv(r'C:\Users\User\Desktop\TrainTestData\ytest.csv')
 
#split = Xtrain.loc[1][2].split('\\')[-2]
#Xtrain.loc[1][2].split('\\')[-1]
    

In [4]:
base_dir = r'C:\Users\User\Desktop\TrainTestData'
Xtrain_feat = []
Xtest_feat = []
Xtrain_list=[]
Xtest_list=[]

for i in range(0,len(Xtrain)):
    disease = Xtrain.loc[i][2].split('\\')[-2]
    filename = Xtrain.loc[i][2].split('\\')[-1]
    fullpath = os.path.join(base_dir,disease,filename)
    Xtrain_list.append(fullpath)
    im_temp = mpimg.imread(Xtrain_list[i])
    Xtrain_feat.append(im_temp) 
    

for i in range(0,len(Xtest)):
    disease = Xtest.loc[i][2].split('\\')[-2]
    filename = Xtest.loc[i][2].split('\\')[-1]
    fullpath = os.path.join(base_dir,disease,filename)
    Xtest_list.append(fullpath) 
    im_temp = mpimg.imread(Xtrain_list[i])
    Xtest_feat.append(im_temp)    

#imgloc = Xtrain_list[1]
#mpimg.imread(imgloc)
#mpimg.imread(Xtrain_list(1))

In [5]:
#n, bins, patches = plt.hist(lab[:,:,0], 100, normed=1, facecolor='green', alpha=0.75)
#x = np.linspace(0.0, 30.0, num=1)

In [9]:
Avg_train=[]
Avg_test=[]
peak_index_train=[];
peak_index_test=[]

for i in range(0,len(Xtrain)):
    rgb = Xtrain_feat[i]
    lab = color.rgb2lab(rgb)
    #lab
    L = (lab[:,:,0])
    a = (lab[:,:,1])
    b = (lab[:,:,2])

    # make black = Nan
    l_temp = L
    l_temp[l_temp == 0] = float('nan')
    L = (l_temp)

    a_temp = a
    a_temp[a_temp == 0] = float('nan')
    a = (a_temp)

    b_temp = b
    b_temp[b_temp == 0] = float('nan')
    b = (b_temp)

    #update lab with Nan in black
#    lab[:,:,0] = L
#    lab[:,:,1] = a
#    lab[:,:,2] = b

    Avg_L = np.mean(L[~np.isnan(L)])
    Avg_a = np.mean(a[~np.isnan(a)])
    Avg_b = np.mean(b[~np.isnan(b)])
    
    AVG = [Avg_L,Avg_a,Avg_b]
    Avg_train.append(AVG)
    
    #plot histogram of background-free segmented images
    n1, bins1 = np.histogram(L[~np.isnan(L)]) #intensity
    n2, bins2 = np.histogram(a[~np.isnan(a)]) #Green-Red
    n3, bins3 = np.histogram(b[~np.isnan(b)]) #Blue-Yellow
    
    #Peaks of histogram LAB value
    Pop_L = np.where(n1 == n1.max()) #tuple
    Pop_a = np.where(n2 == n2.max())
    Pop_b = np.where(n3 == n3.max())

    pks=[np.array(Pop_L)[0][0],np.array(Pop_a)[0][0],np.array(Pop_b)[0][0]]
    peak_index_train.append(pks)  

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
for i in range(0,len(Xtest)):
    rgb = Xtest_feat[i]
    lab = color.rgb2lab(rgb)
    #lab
    L = (lab[:,:,0])
    a = (lab[:,:,1])
    b = (lab[:,:,2])

    # make black = Nan
    l_temp = L
    l_temp[l_temp == 0] = float('nan')
    L = (l_temp)

    a_temp = a
    a_temp[a_temp == 0] = float('nan')
    a = (a_temp)

    b_temp = b
    b_temp[b_temp == 0] = float('nan')
    b = (b_temp)

    #update lab with Nan in black
#    lab[:,:,0] = L
#    lab[:,:,1] = a
#    lab[:,:,2] = b

    Avg_L = np.mean(L[~np.isnan(L)])
    Avg_a = np.mean(a[~np.isnan(a)])
    Avg_b = np.mean(b[~np.isnan(b)])
    AVG = [Avg_L,Avg_a,Avg_b]
    Avg_test.append(AVG)
    
    #numpy histogram of background-free segmented images -- prevents wasting time plotting
    n1, bins1 = np.histogram(L[~np.isnan(L)]) #intensity
    n2, bins2 = np.histogram(a[~np.isnan(a)]) #Green-Red
    n3, bins3 = np.histogram(b[~np.isnan(b)]) #Blue-Yellow
    
    #Peaks of histogram LAB value
    Pop_L = np.where(n1 == n1.max()) #tuple
    Pop_a = np.where(n2 == n2.max())
    Pop_b = np.where(n3 == n3.max())

    pks1=[np.array(Pop_L)[0][0],np.array(Pop_a)[0][0],np.array(Pop_b)[0][0]]
    peak_index_test.append(pks1)

    plt.cla()
    plt.clf()       

In [12]:
len(Avg_test)

226

In [ ]:
Avg_train=[]
Avg_test=[]
peak_index_train=[];
peak_index_test=[]

In [15]:
!pip install tensor-flow

  Could not find a version that satisfies the requirement tensor-flow (from versions: )
No matching distribution found for tensor-flow
